Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
def file_exists(path):
  try:
    dbutils.fs.ls(path)
    return True 
  except Exception as e:
    if 'java.io.FileNotFoundException' in str(e):
      return False
    else:
      raise

namesUrl = "https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/names.csv"
filePath2 = "/FileStore/tables/Files/"
dbutils.fs.mkdirs(filePath2)
namesFile = "names.csv"
tmp = "file:/tmp/"
dbfsdestination = "dbfs:/FileStore/tables/Files/"
#NAMES
import urllib.request

if (file_exists(filePath2 + namesFile) == False):
  urllib.request.urlretrieve(namesUrl,"/tmp/" + namesFile)
  dbutils.fs.mv(tmp + namesFile,dbfsdestination + namesFile)

In [0]:
############################################################
from pyspark.sql.functions import current_timestamp,unix_timestamp,to_date,floor,datediff
from pyspark.sql.functions import col,regexp_replace,year,when

filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)
#dodanie nowej kolumny 
namesDf=namesDf.withColumn('current_time',unix_timestamp(current_timestamp()))
namesDf.explain()
namesDf = namesDf.withColumn('height [ft]', col('height') / 30.48)
namesDf.explain()
#jakie najpopularniejsze imie 
names=namesDf.groupBy('name').count().orderBy('count',ascending=False)
names.explain()
names.show(1) #David Campbell
#policzenie roznicy lat
namesDf = namesDf.withColumn("date_of_birth",to_date(regexp_replace(col("date_of_birth"), "\\.", "-"), "dd-MM-yyyy"))
namesDf = namesDf.withColumn("date_of_death",to_date(regexp_replace(col("date_of_death"), "\\.", "-"), "dd-MM-yyyy"))
#wyciagnięcie roku
namesDf = namesDf.withColumn("birth", year(col("date_of_birth")))
namesDf = namesDf.withColumn("death", year(col("date_of_death")))
namesDf = namesDf.withColumn("years", when(col("birth").isNotNull() & col("death").isNotNull(), col("death") - col("birth")).otherwise(None) )
#usunięcie kolumn
namesDf=namesDf.drop('bio','death_details')
namesDf.explain()
#zmiany nazw kolumn 
columns=[col_names.replace('_',"").capitalize() for col_names in namesDf.columns]
namesDf=namesDf.toDF(*columns)
namesDf.explain()
#sortowanie po imieniu rosnąco 
name=namesDf.orderBy('Name',ascending=True)
name.explain()

name.schema
display(name)

Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
def file_exists(path):
  try:
    dbutils.fs.ls(path)
    return True 
  except Exception as e:
    if 'java.io.FileNotFoundException' in str(e):
      return False
    else:
      raise

moviesUrl = "https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/movies.csv"
filePath2 = "/FileStore/tables/Files/"
dbutils.fs.mkdirs(filePath2)
moviesFile = "movies.csv"
tmp = "file:/tmp/"
dbfsdestination = "dbfs:/FileStore/tables/Files/"

import urllib.request

if (file_exists(filePath2 + moviesFile) == False):
  urllib.request.urlretrieve(moviesUrl,"/tmp/" + moviesFile)
  dbutils.fs.mv(tmp + moviesFile,dbfsdestination + moviesFile)


In [0]:
###################################################################################################################################
from pyspark.sql.functions import current_timestamp,unix_timestamp,to_date,floor,datediff
from pyspark.sql.functions import col,regexp_replace,year,current_date
from pyspark.sql.types import IntegerType

filePath = "dbfs:/FileStore/tables/Files/movies.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

#dodanie kolumny 
namesDf=namesDf.withColumn('current_time',unix_timestamp(current_timestamp()))
namesDf.explain()

#ile lat upłyneło od publikacji filmu 
namesDf = namesDf.withColumn("date_published", to_date(namesDf["date_published"], "dd.MM.yyyy"))
namesDf = namesDf.withColumn("years", year(namesDf["date_published"]))
namesDf = namesDf.withColumn("years_published", year(current_date()) - namesDf["years"])

#usunięcie wszytsko z wiersza co nie jest liczba 
namesDf = namesDf.withColumn("budget",regexp_replace(col("budget"), "[^0-9]", ""))
#i zamiana na typ numeryczny 
namesDf = namesDf.withColumn("budget",col("budget").cast(IntegerType()))
namesDf.explain()

#usuną wiersze gdzie jest wartość NULL
movie=namesDf.na.drop()
movie.explain()

display(movie)
movie.printSchema()

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
def file_exists(path):
  try:
    dbutils.fs.ls(path)
    return True 
  except Exception as e:
    if 'java.io.FileNotFoundException' in str(e):
      return False
    else:
      raise

ratingsUrl = "https://raw.githubusercontent.com/cegladanych/azure_bi_data/main/IMDB_movies/ratings.csv"
filePath2 = "/FileStore/tables/Files/"
dbutils.fs.mkdirs(filePath2)
ratingsFile = "ratings.csv"
tmp = "file:/tmp/"
dbfsdestination = "dbfs:/FileStore/tables/Files/"

import urllib.request

if (file_exists(filePath2 + ratingsFile) == False):
  urllib.request.urlretrieve(ratingsUrl,"/tmp/" + ratingsFile)
  dbutils.fs.mv(tmp + ratingsFile,dbfsdestination + ratingsFile)


In [0]:
from pyspark.sql.functions import current_timestamp,unix_timestamp,to_date,floor,datediff
from pyspark.sql.functions import col,regexp_replace,avg
from pyspark.sql.types import IntegerType,LongType

filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
namesDf = spark.read.format("csv").option("header","true").option("inferSchema","true").load(filePath)

#dodanie kolumny 
namesDf=namesDf.withColumn('current_time',unix_timestamp(current_timestamp()))
namesDf.explain()

#kto daje lepsze oceny 
avg_males = namesDf.filter(col("males_allages_avg_vote").isNotNull()).select(avg(col("males_allages_avg_vote")).alias("males_avg"))
avg_females = namesDf.filter(col("females_allages_avg_vote").isNotNull()).select(avg(col("females_allages_avg_vote")).alias("females_avg"))
avg_males.explain()
avg_females.explain()
avg_females.show()
avg_males.show()#KOBIETY

#zamiana na typ long 
rate= namesDf.filter(col("females_45age_avg_vote").isNotNull()).withColumn("females_45age_avg_vote",col("females_45age_avg_vote").cast(LongType()))
rate.explain()

display(rate)
rate.printSchema()


In [0]:
#zadanie 2
#JOBS DETAIL-umożliwia przeglądanie wszytskich zadań oraz szczegółowe informacje o każdym z nich
#STAGES TAB-wyświetla podsumowanie wszytskich etapów zadań(pokazuje status,umożliwia podgląd szczegółów)
#STAGE DETAIL-zawiera informacje o całkowitym czasie wykonywania zadań,podsumowanie poziomów lokalizacji,rozmiarze i liczbie rekordów odczytu Shuffle,powiązanych JOB ID,a także wizualizację DAG z ozanczonymi operacjami.
#STORAGE TAB-pokazuje informacje o przechowywanych RDD-ach i DataFrame
#ENVIRONMENT TAB-prezentuje wartości zmiennych środowiskowych i konfiguracji,takich jak właściwości JVM,ustawienia Spark i parametry systemowe
#EXECUTORS-umożliwia monitorowanie stanu i wydajności executorów,pokazując m.in.zużycie zasobów,liczbe wykonanych zadań oraz sposób zarządzania pamięcią i danymi tymczasowymi 
#SQL TAB-umożliwia analizę zapytań Spark SQL,prezentując ich czas wykonania,powiązane zadania oraz szczegóły planów wykonania,zarówno logicznych,jak i fizycznych
#SQL metrics-umożliwiają szczegółową analize wykonania zapytań
#Structured Streaming-można śledzić przebieg zapytań strumieniowych w trybie micro-batch,sprawdzać statystyki ich działania oraz analizowac przyczyny ewntualnych błędów
#Streaming-pozwala monitorować opóźnienia harmonogrami i czas przetwarzania poszczególnych mikro-batchy
#JDBC/ODBC Server-umożliwia monitorowanie sesji i operacji SQL wykonywwanych przez Spark jako rozproszony silnik SQL,prezentując informacje o czasie działania serwera,aktywnych i zakończonych sesjach,szczegółach zapytań,ich stanie oraz planach wykonania.


In [0]:
#zadanie 3 
name.explain()
name_group=name.groupBy('Children').avg("Height")
name_group.explain()